# Import

In [ ]:
import xgboost as xgb
from sklearn.metrics import classification_report, roc_auc_score
import pickle
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
# 시각화
import seaborn as sns
import matplotlib.pyplot as plt
!apt-get -qq install fonts-nanum

# 데이터셋 불러오기 및 전처리

In [ ]:
pothole_original = pd.read_csv('/content/drive/MyDrive/DATA_완성/dataset.csv')
pothole = pothole_original

# 경사도 '기타' 전처리
slope_encoding = {
    '0-2%': 0,
    '2-7%': 1,
    '7-15%': 2,
    '15-30%': 3,
    '30-60%': 4,
    '60-100%': 5}
pothole['경사도'] = pothole['경사도'].map(slope_encoding)

# 배수등급 '기타' 전처리
drain_encoding = {
    '매우양호': 5,
    '양호': 4,
    '약간양호': 3,
    '약간불량': 2,
    '불량': 1,
    '매우불량': 0}
pothole['배수등급'] = pothole['배수등급'].map(drain_encoding)

# '기타'인 데이터는 NaN으로 인코딩됨 !
pothole.isna().sum() # NaN - 2533

# 자치구도 반영하기!
gu_encoder = LabelEncoder()
pothole['자치구_enc'] = gu_encoder.fit_transform(pothole['자치구'])

# 채워넣을 데이터 분리
knn_train = pothole[pothole['배수등급'].notna()]
knn_test = pothole[pothole['배수등급'].isna()]

In [ ]:
### 모델 - 경사도
# 모델 fitting
knn_slope = KNeighborsClassifier(n_neighbors = 3)
knn_slope.fit(knn_train[['경도', '위도', '자치구_enc']], knn_train['경사도'])
# prediction
pred_slope = knn_slope.predict(knn_test[['경도', '위도', '자치구_enc']])
pothole.loc[knn_test.index, '경사도'] = pred_slope


### 모델 - 배수등급
# 모델 fitting
knn_drain = KNeighborsClassifier(n_neighbors = 3)
knn_drain.fit(knn_train[['경도', '위도', '자치구_enc']], knn_train['배수등급'])
# prediction
pred_drain = knn_drain.predict(knn_test[['경도', '위도', '자치구_enc']])
pothole.loc[knn_test.index, '배수등급'] = pred_drain

In [ ]:
scaling_vars = ['승용차', '버스', '트럭', '총교통량', '중대형차량 교통량', '평균_건물연령',
                '누적 강수량', '평균 습도', '1년 기온차', '평균 일교차']

scaler = StandardScaler()
pothole[scaling_vars] = scaler.fit_transform(pothole[scaling_vars])

In [ ]:
# 전처리 완료 후 저장
pothole.to_pickle('pothole_예측용.pickle')

# 전처리 완료된 데이터셋 불러오기

In [ ]:
with open(r"pothole_예측용.pickle","rb") as f:
    dataset = pickle.load(f)
dataset

,자치구,행정동,경도,위도,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,...,등록번호,발생일,인구 수,누적 강수량,평균 습도,1년 기온차,평균 일교차,배수등급,경사도,자치구_enc
0,강남구,개포동,127.075036,37.486016,1.0,-0.355737,-0.057007,-0.295055,-0.360729,0.079294,...,NaN,NaN,0.928563,-0.599417,-0.038033,-1.144723,0.668894,4.0,2.0,0
1,강남구,개포동,127.074160,37.485413,1.0,-0.355737,-0.057007,-0.295055,-0.360729,0.079294,...,NaN,NaN,0.928563,-0.599417,-0.038033,-1.144723,0.668894,4.0,4.0,0
2,강남구,개포동,127.067286,37.476065,1.0,-0.214723,-0.852475,-0.443420,-0.275985,-0.923940,...,2.023080e+16,2023-07-03,0.928563,0.508823,-0.135865,0.755091,0.642615,4.0,3.0,0
3,강남구,개포동,127.068451,37.486155,1.0,-0.214723,-0.852475,-0.443420,-0.275985,-0.923940,...,2.023122e+16,2023-11-19,0.928563,0.508823,-0.049247,0.755091,0.832790,3.0,1.0,0
4,강남구,개포동,127.054309,37.486003,2.0,-1.093681,-2.153068,-1.600422,-1.206700,-1.803203,...,2.023040e+16,2023-03-03,0.928563,0.508823,-0.160266,0.755091,0.604751,4.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15475,중랑구,중화동,127.079953,37.598558,6.0,0.082505,0.481667,0.448345,0.129215,0.686994,...,2.023113e+16,2023-10-29,-0.051377,0.532140,-1.373144,0.530717,0.044423,2.0,0.0,24
15476,중랑구,중화동,127.072169,37.593029,6.0,0.247654,1.511118,0.575937,0.342567,1.209434,...,NaN,NaN,-0.051377,-0.566928,-1.347690,-1.273924,-0.186021,2.0,0.0,24
15477,중랑구,중화동,127.078758,37.594663,6.0,0.250991,1.132762,0.455431,0.307704,0.776990,...,NaN,NaN,-0.051377,-0.566928,-1.347690,-1.273924,-0.186021,2.0,0.0,24
15478,중랑구,중화동,127.075799,37.593907,7.0,0.383152,1.323956,0.571288,0.444554,0.809560,...,NaN,NaN,-0.051377,-0.566928,-1.347690,-1.273924,-0.186021,1.0,0.0,24


In [ ]:
dataset['포트홀 존재'] = 1  # 기본값 1로 초기화
dataset.loc[dataset['포트홀 갯수'] == 0, '포트홀 존재'] = 0

# x, y 할당

In [ ]:
#x = dataset.drop(['자치구', '행정동', '경도', '위도', '등록번호', '발생일', '포트홀 갯수', '자치구_enc', '포트홀 존재'], axis=1)

# 기후 관련 컬럼 제거
x = dataset.drop(['자치구', '행정동', '경도', '위도', '등록번호', '발생일', '포트홀 갯수', '자치구_enc', '포트홀 존재',
                  '누적 강수량', '평균 습도', '1년 기온차', '평균 일교차'], axis=1)
y = dataset['포트홀 존재']

In [ ]:
x

,차선수,승용차,버스,트럭,총교통량,중대형차량 교통량,평균_건물연령,인구 수,배수등급,경사도
0,1.0,-0.355737,-0.057007,-0.295055,-0.360729,0.079294,-0.363365,0.928563,4.0,2.0
1,1.0,-0.355737,-0.057007,-0.295055,-0.360729,0.079294,-0.363365,0.928563,4.0,4.0
2,1.0,-0.214723,-0.852475,-0.443420,-0.275985,-0.923940,-0.363365,0.928563,4.0,3.0
3,1.0,-0.214723,-0.852475,-0.443420,-0.275985,-0.923940,-0.363365,0.928563,3.0,1.0
4,2.0,-1.093681,-2.153068,-1.600422,-1.206700,-1.803203,-0.363365,0.928563,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...
15475,6.0,0.082505,0.481667,0.448345,0.129215,0.686994,0.189077,-0.051377,2.0,0.0
15476,6.0,0.247654,1.511118,0.575937,0.342567,1.209434,0.189077,-0.051377,2.0,0.0
15477,6.0,0.250991,1.132762,0.455431,0.307704,0.776990,0.189077,-0.051377,2.0,0.0
15478,7.0,0.383152,1.323956,0.571288,0.444554,0.809560,0.189077,-0.051377,1.0,0.0


# train, test 분리

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape)

(12384, 10) (3096, 10)


# SMOTE

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [ ]:
X_train_resampled.to_csv('X_train.csv', index=False)

In [ ]:
print("원래 학습 데이터 클래스 분포:\n", y_train.value_counts(), '\n')
print("SMOTE 적용 후 클래스 분포:\n", pd.Series(y_train_resampled).value_counts())

원래 학습 데이터 클래스 분포:
 포트홀 존재
0    6782
1    5602
Name: count, dtype: int64 

SMOTE 적용 후 클래스 분포:
 포트홀 존재
1    6782
0    6782
Name: count, dtype: int64


# 모델 적합, 평가

In [ ]:
# 모델 평가
def get_clf_eval(y_test, pred=None, pred_proba=None):

    cm = confusion_matrix(y_test, pred)
    tn, fp, fn, tp = cm.ravel()  # 혼동 행렬 해체

    print('오차행렬 \n', cm)
    print('정확도 :', accuracy_score(y_test, pred))
    print('정밀도 : ',precision_score(y_test, pred))
    print('재현율 :', recall_score(y_test, pred))
    print('f1 score :', f1_score(y_test, pred))
    print('roc auc score :', roc_auc_score(y_test, pred_proba))

    # False Negative Rate. 실제 포트홀이 있는데 없다고 예측한 경우
    fnr = fn / (fn + tp)
    print(f'False Negative Rate (FNR): {fnr}')

def get_model_train_eval(model, X_train = None, X_test = None, y_train=None, y_test=None):
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    pred_proba = model.predict_proba(X_test)[:,1]
    get_clf_eval(y_test, pred, pred_proba)
    return pred_proba

## XGBoost

In [ ]:
# 기후 제거. 기본 모델.
model = xgb.XGBClassifier(random_state=42)

pred_proba = get_model_train_eval(model, X_train_resampled, X_test, y_train_resampled, y_test)

오차행렬 
 [[1338  358]
 [ 347 1053]]
정확도 : 0.7722868217054264
정밀도 :  0.7462792345854005
재현율 : 0.7521428571428571
f1 score : 0.7491995731056563
roc auc score : 0.8434423433288409
False Negative Rate (FNR): 0.24785714285714286


# Hyperparameter Tuning

In [ ]:
# 하이퍼파라미터 후보군 설정
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [0.001, 0.005, 0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'min_child_weight': [1, 2, 3, 4, 5],
    'gamma': [0, 0.1, 0.3, 0.5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'scale_pos_weight': [1, 2, 3]  # 불균형 데이터일 경우 유용
}

model = xgb.XGBClassifier(random_state=42)

# RandomizedSearchCV 설정
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=1000,
    scoring='f1',
    n_jobs=-1,
    cv=5,
    verbose=1,
    random_state=42
)

# 모델 학습 및 튜닝
random_search.fit(X_train_resampled, y_train_resampled)

# 결과 출력
print("Best Parameters:", random_search.best_params_)
print("Best CV Accuracy:", random_search.best_score_)

# 최적 모델로 예측 및 평가
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))

cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = cm.ravel()  # 혼동 행렬 해체

print('오차행렬 \n', cm)
print('정확도 :', accuracy_score(y_test, y_pred))
print('정밀도 : ',precision_score(y_test, y_pred))
print('재현율 :', recall_score(y_test, y_pred))
print('f1 score :', f1_score(y_test, y_pred))
print('roc auc score :', roc_auc_score(y_test, pred_proba))

# False Negative Rate. 실제 포트홀이 있는데 없다고 예측한 경우
fnr = fn / (fn + tp)
print(f'False Negative Rate (FNR): {fnr}')

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits
Best Parameters: {'subsample': 0.8, 'scale_pos_weight': 2, 'n_estimators': 300, 'min_child_weight': 4, 'max_depth': 10, 'learning_rate': 0.05, 'gamma': 0.3, 'colsample_bytree': 0.8}
Best CV Accuracy: 0.7854688667656067
Test Accuracy: 0.7522609819121447
오차행렬 
 [[1185  511]
 [ 256 1144]]
정확도 : 0.7522609819121447
정밀도 :  0.691238670694864
재현율 : 0.8171428571428572
f1 score : 0.7489361702127659
roc auc score : 0.8434423433288409
False Negative Rate (FNR): 0.18285714285714286


# Final 모델

In [ ]:
model = xgb.XGBClassifier(subsample=0.6,
                          scale_pos_weight= 2,
                          n_estimators= 400,
                          min_child_weight= 3,
                          max_depth= 8,
                          learning_rate= 0.05,
                          gamma= 0.1,
                          colsample_bytree= 0.8,
                          random_state=42)
pred_proba = get_model_train_eval(model, X_train_resampled, X_test, y_train_resampled, y_test)

오차행렬 
 [[1176  520]
 [ 244 1156]]
정확도 : 0.7532299741602068
정밀도 :  0.6897374701670644
재현율 : 0.8257142857142857
f1 score : 0.7516254876462939
roc auc score : 0.8535211000673855
False Negative Rate (FNR): 0.1742857142857143


In [ ]:
pred_proba[:30]

array([0.03443918, 0.5295496 , 0.693969  , 0.76384586, 0.41232562,
       0.52332234, 0.49969894, 0.7289811 , 0.965679  , 0.23123744,
       0.03491516, 0.8598307 , 0.56108093, 0.17499644, 0.15877001,
       0.8560248 , 0.43234587, 0.02563023, 0.92067724, 0.9713283 ,
       0.45529124, 0.94638574, 0.9986099 , 0.6045762 , 0.97534114,
       0.6871595 , 0.75336784, 0.9615072 , 0.1516675 , 0.04926291],
      dtype=float32)

In [ ]:
importances = model.feature_importances_
importance_df = pd.DataFrame({
    'Feature': X_train_resampled.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

print(importance_df)

     Feature  Importance
4       총교통량    0.146566
0        차선수    0.127642
7       인구 수    0.121365
3         트럭    0.107623
6    평균_건물연령    0.087603
2         버스    0.086881
1        승용차    0.083102
9        경사도    0.080241
8       배수등급    0.079722
5  중대형차량 교통량    0.079254


In [ ]:
with open("model.pickle", "wb") as fw:
    pickle.dump(model, fw)

# 도로별 그룹화

In [ ]:
# 전체 데이터로 모델 재학습
final_model = xgb.XGBClassifier(subsample=0.6,
                          scale_pos_weight= 2,
                          n_estimators= 400,
                          min_child_weight= 3,
                          max_depth= 8,
                          learning_rate= 0.05,
                          gamma= 0.1,
                          colsample_bytree= 0.8,
                          random_state=42)
final_model.fit(x, y)

# 전체 데이터에 대한 예측 확률
result = dataset.copy()

result['예측값'] = final_model.predict_proba(x)[:, 1]

In [ ]:
print(result[['차선수','총교통량','중대형차량 교통량','포트홀 존재','예측값']].head(5))

   차선수      총교통량  중대형차량 교통량  포트홀 존재       예측값
0  1.0 -0.360729   0.079294       0  0.225296
1  1.0 -0.360729   0.079294       0  0.114406
2  1.0 -0.275985  -0.923940       1  0.997523
3  1.0 -0.275985  -0.923940       1  0.997822
4  2.0 -1.206700  -1.803203       1  0.845239


In [ ]:
road_groups = result.groupby(['차선수', '총교통량', '중대형차량 교통량']).agg(
    actual_count=('포트홀 존재', 'sum'),
    predicted_sum=('예측값', 'sum'),
    count=('포트홀 존재', 'size')    # number of records in this group
).reset_index()

In [ ]:
road_groups

,차선수,총교통량,중대형차량 교통량,actual_count,predicted_sum,count
0,1.0,-4.058980,3.265334,0,0.056748,1
1,1.0,-4.047362,2.598037,0,0.186735,1
2,1.0,-3.866221,4.360285,0,0.038354,1
3,1.0,-3.828758,3.388194,0,0.037741,1
4,1.0,-3.787499,3.986006,0,0.037551,1
...,...,...,...,...,...,...
6910,10.0,1.839362,-0.728417,1,0.953053,1
6911,10.0,1.912237,-0.708553,4,3.783004,4
6912,10.0,1.983514,-0.839959,2,1.890352,2
6913,11.0,0.353396,0.547864,2,1.650548,2


In [ ]:
# Calculate prediction error per road (difference and absolute difference)
road_groups['error'] = road_groups['predicted_sum'] - road_groups['actual_count']
road_groups['abs_error'] = road_groups['error'].abs()

In [ ]:
road_groups.sort_values('actual_count', ascending=False, inplace=True)

In [ ]:
print(road_groups[['차선수','총교통량','중대형차량 교통량','actual_count','predicted_sum','error']].head(10))

      차선수      총교통량  중대형차량 교통량  actual_count  predicted_sum     error
735   1.0 -0.275985  -0.923940           729     726.824890 -2.175110
1952  2.0  0.184210   2.476383            24      24.309074  0.309074
1840  2.0 -0.084519   2.206816            22      22.380312  0.380312
5665  6.0 -0.163658  -0.126438            20      20.043438  0.043438
5559  6.0 -0.285446   0.625307            19      19.343739  0.343739
2140  2.0  0.739204  -0.680183            17      16.718311 -0.281689
2877  3.0  2.041829  -0.638758            17      16.907480 -0.092520
2682  3.0  0.511929   1.296202            17      16.845243 -0.154757
5802  6.0 -0.048510  -0.104162            17      16.866451 -0.133549
3965  4.0 -0.274865   0.103782            16      16.534910  0.534910
